# Problem 2

In [1]:
import numpy as np
import networkx as nx 

from itertools import combinations
from tqdm import tqdm

### (1) Find the densest subgraph using the Charikar greedy algorithm

In [2]:
nodes = list(range(1,11))

edges = [
    (1,2),
    (1,3),
    (1,4),
    (1,7),
    (2,3),
    (2,4),
    (2,5),
    (3,4),
    (3,7),
    (4,5),
    (5,6),
    (5,9),
    (6,8),
    (6,9),
    (6,10),
    (7,8),
    (7,10),
    (8,9),
    (8,10),
    (9,10)
]

In [3]:
# Adjacency matrix

A = np.zeros((10,10))

for e in edges:
    A[e[0] - 1, e[1] - 1] = 1
    A[e[1] - 1, e[0] - 1] = 1

# Degree matrix

D = np.diag(np.sum(A, axis = 0))

# Laplacian matrix

L = D - A

# Adjacency lists

adj_lists = {v:[] for v in nodes}

for e in edges:
    adj_lists[e[0]].append(e[1])
    adj_lists[e[1]].append(e[0])

In [4]:
adj_lists

{1: [2, 3, 4, 7],
 2: [1, 3, 4, 5],
 3: [1, 2, 4, 7],
 4: [1, 2, 3, 5],
 5: [2, 4, 6, 9],
 6: [5, 8, 9, 10],
 7: [1, 3, 8, 10],
 8: [6, 7, 9, 10],
 9: [5, 6, 8, 10],
 10: [6, 7, 8, 9]}

In [5]:
def Greedy_Densest_Subgraph(
        adj_lists:dict
        ) -> list:
    
    S = list(adj_lists.keys())

    S_G = list(adj_lists.keys())

    f_SG = 0.5*sum(map(lambda x: len(adj_lists[x]), adj_lists))/len(S)

    print(f_SG)
    
    while len(S) > 1:

        v = min(adj_lists, key = lambda k: len(adj_lists[k]))

        S.remove(v)

        adj_lists = {u : [node for node in adj_lists[u] if node != v] for u in list(adj_lists.keys()) if u != v}

        print(adj_lists)
        
        f_S = 0.5*sum(map(lambda x: len(adj_lists[x]), adj_lists))/len(S)
        
        print(f_S)
        
        if f_S >= f_SG:
               
               f_SG = f_S

               S_G = S

    return S_G

In [6]:
Greedy_Densest_Subgraph(adj_lists)

2.0
{2: [3, 4, 5], 3: [2, 4, 7], 4: [2, 3, 5], 5: [2, 4, 6, 9], 6: [5, 8, 9, 10], 7: [3, 8, 10], 8: [6, 7, 9, 10], 9: [5, 6, 8, 10], 10: [6, 7, 8, 9]}
1.7777777777777777
{3: [4, 7], 4: [3, 5], 5: [4, 6, 9], 6: [5, 8, 9, 10], 7: [3, 8, 10], 8: [6, 7, 9, 10], 9: [5, 6, 8, 10], 10: [6, 7, 8, 9]}
1.625
{4: [5], 5: [4, 6, 9], 6: [5, 8, 9, 10], 7: [8, 10], 8: [6, 7, 9, 10], 9: [5, 6, 8, 10], 10: [6, 7, 8, 9]}
1.5714285714285714
{5: [6, 9], 6: [5, 8, 9, 10], 7: [8, 10], 8: [6, 7, 9, 10], 9: [5, 6, 8, 10], 10: [6, 7, 8, 9]}
1.6666666666666667
{6: [8, 9, 10], 7: [8, 10], 8: [6, 7, 9, 10], 9: [6, 8, 10], 10: [6, 7, 8, 9]}
1.6
{6: [8, 9, 10], 8: [6, 9, 10], 9: [6, 8, 10], 10: [6, 8, 9]}
1.5
{8: [9, 10], 9: [8, 10], 10: [8, 9]}
1.0
{9: [10], 10: [9]}
0.5
{10: []}
0.0


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

### (3) Show that Cheeger's inequality hold for this graph

Given the second smallest eigenvalue $\lambda_2$ of the graph laplacian, Cheeger's inequality states that the graph conductance $\phi_G$ is bounded as follows:

$$ \frac{\lambda_2}{2} \leq \phi_G \leq \lambda_2 $$

where the conductance is defined as the minimum among the conductance of all cuts, where a cut conductance is the cardinality of the cut over its volume as the minimum between the sum of the degrees of the nodes withing one of the two partitions:

$$ \phi_G = \underset{C1 \subset V, C2 = V \setminus C1}{\mathrm{min}} \ \ \frac{|E(C1,C2)|}{\min\{ \sum_{i \in C1} d_i, \sum_{i \in C2} d_i\}} $$

Since the given graph is quite small, we can compute its conductance explicitely:

In [7]:
cuts_conductance = {}

for m in tqdm(range(1,6)):

    cuts = list(combinations(nodes, m))

    for cut in cuts:

        C1 = cut

        C2 = [node for node in nodes if node not in cut]

        E_C1C2 = len([e for e in edges if (e[0] in C1 and e[1] in C2) or (e[1] in C1 and e[0] in C2)])

        sum_d1 = sum(map(len, 
                         dict(
                             filter(
                                 lambda item: item[0] not in C1, adj_lists.items())).values()))
        
        sum_d2 = sum(map(len, 
                         dict(
                             filter(
                                 lambda item: item[0] not in C2, adj_lists.items())).values()))
        
        if (sum_d1 - E_C1C2 != 0) and (sum_d2 - E_C1C2 != 0):

            cuts_conductance[C1] = E_C1C2 / min((sum_d1 - E_C1C2, sum_d2 - E_C1C2))
                

100%|██████████| 5/5 [00:00<00:00, 833.66it/s]


In [8]:
phi_G = min(cuts_conductance.values())

In [9]:
phi_G

0.25

Now to check if Cheeger's inequality holds we need the second smallest eigenvalue of the normalized laplacian matrix: since our graph is $d$-regular, with $d = 4$ we have that 
$$ \mathcal{L} = \mathbb{I} - \frac{1}{d}A $$

In [10]:
L_ = np.eye(L.shape[0]) - A/4

In [11]:
lambdas, U = np.linalg.eig(L_)

In [12]:
lambda_2 = np.sort(np.real(lambdas))[1]

In [13]:
lambda_2

0.2500000000000001

In [14]:
(lambda_2/2 <= phi_G) & (phi_G <= np.sqrt(2*lambda_2))

True

__________